In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [2]:
data = pd.read_json('Brisbane_CityBike.json')
print("Dataset shape ", data.shape)
data.head()

Dataset shape  (149, 5)


,address,latitude,longitude,name,number
0,Lower River Tce / Ellis St,-27.482279,153.028723,122 - LOWER RIVER TCE / ELLIS ST,122
1,Main St / Darragh St,-27.470590,153.036046,91 - MAIN ST / DARRAGH ST,91
2,Sydney St Ferry Terminal / Park,-27.474531,153.042728,88 - SYDNEY ST FERRY TERMINAL / PARK,88
3,Browne St / James St,-27.461881,153.046986,75 - BROWNE ST / JAMES ST,75
4,Kurilpa Point / Montague Rd,-27.469658,153.016696,98 - KURILPA POINT / MONTAGUE RD,98


In [3]:
#def transform(s):
#    tokens = s.split('-', 1)
#    if(len(tokens)>1):
#        return tokens[1].strip()
#    return None
#
#data['corner_1'], data['corner_2'] = data['name'].str.split('/', 1).str
#
#data['corner_1'] = data['corner_1'].apply( lambda x: transform(x))
#
#data['corner_2'] = data['corner_2'].str.strip()
#
#data.head()

## HDBSCAN clustering on coordinates

In [4]:
#Apply haversine distance on RADIANS
#http://hdbscan.readthedocs.io/en/latest/faq.html?highlight=lat%20lon%27#q-haversine-metric-is-not-clustering-my-lat-lon-data-correctly
import hdbscan
points = list(zip(data.latitude, data.longitude))
rads = np.radians(points)
clusterer = hdbscan.HDBSCAN(min_cluster_size=4, metric='haversine')
cluster_label_haversines = clusterer.fit_predict(rads)
print("Extracted Clusters: ", len(set(cluster_label_haversines)))

Extracted Clusters:  6


In [5]:
#Apply euclidean distance on original data
import hdbscan
points = list(zip(data.latitude, data.longitude))

clusterer = hdbscan.HDBSCAN(min_cluster_size=3, metric='euclidean')
cluster_labels_euclidean = clusterer.fit_predict(points)
print("Extracted Clusters: ", len(set(cluster_labels_euclidean)))

Extracted Clusters:  7


## Data Visualization

In [6]:
c_lat =  data["latitude"].mean()
c_lon = data["longitude"].mean()

cluster_labels = cluster_labels_euclidean 

In [7]:

palette = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99']

classes = range(-1, len(palette) -1)
colors = dict(zip(classes, palette))

c_array = [colors[x]for x in cluster_labels]

from gmplot import gmplot
gmap = gmplot.GoogleMapPlotter(c_lat, c_lon, 14)
for c in classes:
    index = [i for i,v in enumerate(cluster_labels) if cluster_labels[i]==c]
    subset = data.iloc[index]
    if(c==-1):
        color = "#000000"
    else:
        color = colors[c]
    gmap.scatter(subset["latitude"], subset["longitude"], c = color, size=60, marker=False, face_color= "#FF0000", face_alpha= 1)
    #gmap.heatmap(data["latitude"], data["longitude"], radius=30)
    gmap.draw("my_map2.html")

#open the html page with a browser    

In [8]:
gmap.heatmap(data["latitude"], data["longitude"], radius=50)
gmap.draw("my_map3.html")

In [9]:

#filename = "my_map3"
#
#from selenium import webdriver    
#from xvfbwrapper import Xvfb
#d=Xvfb(width=400,height=400)
#d.start()
#browser=webdriver.Firefox()
#url=filename+ ".html"
#browser.get(url)
#destination=filename +".jpg"
#if browser.save_screenshot(destination):
#    print("File saved in the destination filename")
#browser.quit()